In [ ]:
#Parte 1, Ejercicio 1

# Uilizamos la base de datos del TP 3 para ampliar la información que contiene  "Respondieron"
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

df = pd.read_excel(r"C:\Users\ldtes\baseTP3.xlsx")
df = df.drop(columns=['Unnamed: 0','cluster_k2', 'cluster_k4', 'cluster_k10', 'cluster_k2_label'])
no_respondieron = df[df['ESTADO'] == "Entrevista individual no realizada (no respuesta al Cuestionario Individual)"]
respondieron = df[df['ESTADO'] != "Entrevista individual no realizada (no respuesta al Cuestionario Individual)"]
column_names = df.columns.tolist()

In [ ]:
from scipy import stats
import numpy as np
from scipy.stats import ttest_ind
import ISLP as islp
from ISLP.models import ModelSpec
import statsmodels.api as sm
#Creamos la variable dummy correspondiente a "Desocupado" 
respondieron['desocupado'] = (respondieron['ESTADO'] == 'Desocupado').astype(int)

#Añadimos otras dummies para incorporar más información de nuestra base datos al modelo
#Dummies incorporadas
respondieron['mujer'] = respondieron['Sexos Relevados'].map({"Mujer": 1, 'Varón': 0})
respondieron['lee_dummy'] = respondieron['Sabe leer y escribir'].map({'Sí': 1, 'No': 2})
respondieron['vp_dummy'] = respondieron['Tiene Vacaciones Pagas'].map({'Sí': 1, 'No': 2})
respondieron["salario_semanal"]=respondieron["salario_semanal"].round(2)

#Ajustamos la variable "Ingreso Per cápita familiar" a la inflación con un deflactor del PBI calculado en el TP3  
df.loc[df['ANO4'] == 2004, 'Ingreso Per Cápita Familiar'] *= 882.22

#Designamos la variable dependiente e independiente
y=respondieron[["desocupado"]]
x=respondieron[['ANO4', 'Ingreso Per Cápita Familiar', 'Años al momento de la encuesta', 'DECCFR', 'IDECCFR',
 'RDECCFR', 'GDECCFR', 'PDECCFR', 'ADECCFR', 'edad2', 'PP3F_TOT', 'PP3E_TOT', 'P21', 'educ', 'salario_semanal', 'horas_trab', 'mujer',"lee_dummy" ,'vp_dummy']]

#Reemplazamos todos los NaN por ceros y agregamos la columna de unos
x.replace([np.nan, np.inf, -np.inf], 0, inplace=True)
x['intercepto'] = 1

In [226]:
#Dividimos la información en datos de entrenamiento y testeo:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=444)
x.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

# Evaluamos el test-t y el p-valor para cada grupo
#El test-t contrasta las medias entre la muestra de entrenamiento y la de testeo.
t_test_x = stats.ttest_ind(x_train, x_test)
t_test_y = stats.ttest_ind(y_train, y_test)

#Calculamos datos :

# Para x_train
print("Número de observaciones:", x_train.shape[0])
print("Media:\n", x_train.mean())
print("Desvío estándar:\n", x_train.std())

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_61076\1293336932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.replace([np.nan, np.inf, -np.inf], 0, inplace=True)
C:\Users\ldtes\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:551: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, axis=axis, **kwds)


Número de observaciones: 10259
Media:
ANO4                               2013.538941
Ingreso Per Cápita Familiar       76130.988888
Años al momento de la encuesta       35.605615
DECCFR                                6.404425
IDECCFR                               0.000000
RDECCFR                               6.212691
GDECCFR                               6.289794
PDECCFR                               0.000000
ADECCFR                               6.255678
edad2                              1792.587387
PP3F_TOT                              1.187640
PP3E_TOT                             18.116678
P21                               57943.690223
educ                                  6.406472
salario_semanal                    4539.429051
horas_trab                           15.370406
mujer                                 0.525685
lee_dummy                             1.034994
vp_dummy                              0.437372
intercepto                            1.000000
dtype: float64
Desvío 

In [227]:
# Para x_test
print("Número de observaciones:", x_test.shape[0])
print("Media:\n", x_test.mean())
print("Desvío estándar:\n", x_test.std())

Número de observaciones: 4398
Media:
 ANO4                               2013.627103
Ingreso Per Cápita Familiar       79674.263959
Años al momento de la encuesta       35.714416
DECCFR                                6.450659
IDECCFR                               0.000000
RDECCFR                               6.256025
GDECCFR                               6.335834
PDECCFR                               0.000000
ADECCFR                               6.307640
edad2                              1795.008640
PP3F_TOT                              0.903365
PP3E_TOT                             17.851523
P21                               58352.464757
educ                                  6.452706
salario_semanal                    4329.288538
horas_trab                           15.537062
mujer                                 0.527513
lee_dummy                             1.040473
vp_dummy                              0.428831
intercepto                            1.000000
dtype: float64
Desvío 

In [228]:
# Para y_train
print("Número de observaciones:", y_train.shape[0])
print("Media:", y_train.mean())
print("Desvío estándar:", y_train.std())

Número de observaciones: 10259
Media: desocupado    0.057413
dtype: float64
Desvío estándar: desocupado    0.232641
dtype: float64


In [229]:
# Para y_test
print("Número de observaciones:", y_test.shape[0])
print("Media:", y_test.mean())
print("Desvío estándar:", y_test.std())

Número de observaciones: 4398
Media: desocupado    0.056844
dtype: float64
Desvío estándar: desocupado    0.231571
dtype: float64


In [230]:
#Test T
print("t_test_x:", t_test_x)
print("t_test_y:", t_test_y)
print("p-valor del t_test_x:", t_test_x.pvalue)
print("p-valor del t_test_y:", t_test_y.pvalue)
#Con esta información armamos nuestra tabla de diferencia de medias

t_test_x: TtestResult(statistic=array([-0.48957873, -0.70548367, -0.26388611, -0.67003439,         nan,
       -0.6212184 , -0.66025545,         nan, -0.74658084, -0.07094346,
        0.63278981,  0.25043539, -0.10167614, -0.44242576,  0.8192999 ,
       -0.40807656, -0.20308217, -1.04041051,  0.68526571,         nan]), pvalue=array([0.62443936, 0.4805205 , 0.79187139, 0.50284643,        nan,
       0.5344656 , 0.50910029,        nan, 0.45532859, 0.94344372,
       0.52688079, 0.80225417, 0.91901513, 0.65818767, 0.41262865,
       0.68322343, 0.83907362, 0.29816638, 0.49318711,        nan]), df=array([14655., 14655., 14655., 14655., 14655., 14655., 14655., 14655.,
       14655., 14655., 14655., 14655., 14655., 14655., 14655., 14655.,
       14655., 14655., 14655., 14655.]))
t_test_y: TtestResult(statistic=array([0.13588427]), pvalue=array([0.89191464]), df=array([14655.]))
p-valor del t_test_x: [0.62443936 0.4805205  0.79187139 0.50284643        nan 0.5344656
 0.50910029        nan 0.4

In [231]:
#Parte 1, Ejercicio 2: 

#Debemos conformar nuestro modelo de regresión únicamente para los ocupados de la EPH

# Establecemos primero una base de datos que solo comprenda información para los ocupados de la EPH en nuestra región
ocu=respondieron[respondieron["ESTADO"] == "Ocupado"]

#Creamos la columna de unos
ocu['intercepto'] = 1
ocu.rename(columns={'Años al momento de la encuesta': 'edad'}, inplace=True)
ocu

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_61076\605685891.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ocu['intercepto'] = 1
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_61076\605685891.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ocu.rename(columns={'Años al momento de la encuesta': 'edad'}, inplace=True)


,Sexos Relevados,REGION,ANO4,Nivel Educativo,Estado Civil,Cobertura Médica,Categoría de Inactividad,Ingreso Per Cápita Familiar,edad,Sabe leer y escribir,...,PP3E_TOT,P21,educ,salario_semanal,horas_trab,desocupado,mujer,lee_dummy,vp_dummy,intercepto
0,Mujer,Gran Buenos Aires,2004,Primaria Completa,Casado,Mutual/Prepaga/Servicio de emergencia,NaN,7.357143e+02,59.0,Sí,...,50.0,NaN,6.0,NaN,50.0,0,1,1.0,NaN,1
1,Mujer,Gran Buenos Aires,2004,Primaria Completa,Casado,Mutual/Prepaga/Servicio de emergencia,NaN,3.416333e+03,46.0,Sí,...,18.0,250.0,6.0,5513.88,18.0,0,1,1.0,NaN,1
14,Mujer,Gran Buenos Aires,2004,Primaria Completa,Casado,No paga ni le descuentan,NaN,NaN,48.0,Sí,...,NaN,NaN,6.0,NaN,0.0,0,1,1.0,NaN,1
17,Mujer,Gran Buenos Aires,2004,Primaria Completa,Casado,No paga ni le descuentan,NaN,1.666667e+01,50.0,Sí,...,9.0,NaN,6.0,NaN,9.0,0,1,1.0,NaN,1
19,Mujer,Gran Buenos Aires,2004,Primaria Completa,Casado,No paga ni le descuentan,NaN,2.666667e+01,54.0,Sí,...,28.0,NaN,6.0,NaN,28.0,0,1,1.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14691,Varón,Gran Buenos Aires,2024,Superior Universitaria Incompleta,unido?,Obra social (incluye PAMI),NaN,8.050000e+05,50.0,Sí,...,50.0,800000.0,18.0,20000.00,50.0,0,0,1.0,1.0,1
14692,Varón,Gran Buenos Aires,2024,Superior Universitaria Incompleta,unido?,Obra social (incluye PAMI),NaN,1.285000e+06,37.0,Sí,...,40.0,380000.0,14.0,9500.00,40.0,0,0,1.0,1.0,1
14695,Varón,Gran Buenos Aires,2024,Superior Universitaria Incompleta,viudo/a?,Obra social (incluye PAMI),NaN,NaN,61.0,Sí,...,40.0,535000.0,15.0,13375.00,40.0,0,0,1.0,1.0,1
14696,Varón,Gran Buenos Aires,2024,Superior Universitaria Incompleta,viudo/a?,Obra social (incluye PAMI),NaN,NaN,68.0,Sí,...,20.0,NaN,15.0,NaN,20.0,0,0,1.0,NaN,1


In [177]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
# Planteamos nuestra matriz X
X_ocu=ocu[["edad","edad2","intercepto","educ","mujer","vp_dummy", "lee_dummy"]]
Y_ocu=ocu[["salario_semanal"]]

#Dividimos los datos correspondientes en entrenamiento y testeo

x_train_ocu,x_test_ocu,y_train_ocu,y_test_ocu=train_test_split(X_ocu,Y_ocu,test_size=0.3, random_state=444)



In [216]:
#Con esta información vamos a armar nuestra tabla de diferencia de medias:
#Regresión de salario_semanal en edad

X_1 = x_train_ocu[["edad", "intercepto"]]
X_1.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

Y_1 = y_train_ocu["salario_semanal"]
Y_1.replace([np.nan, np.inf, -np.inf], 0, inplace=True)
model = sm.OLS(Y_1, X_1)
results = model.fit()
print(results.summary())


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_61076\3753684705.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_1.replace([np.nan, np.inf, -np.inf], 0, inplace=True)


                            OLS Regression Results                            
Dep. Variable:        salario_semanal   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     20.39
Date:                Fri, 30 May 2025   Prob (F-statistic):           6.49e-06
Time:                        20:55:05   Log-Likelihood:                -48323.
No. Observations:                4412   AIC:                         9.665e+04
Df Residuals:                    4410   BIC:                         9.666e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
edad          67.1859     14.880      4.515      0.0

In [179]:
#salario_semanal en edad y edad2
X_2 = x_train_ocu[["edad", "edad2", "intercepto"]]
X_2.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

Y_2 = y_train_ocu["salario_semanal"]
Y_2.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

model = sm.OLS(Y_2, X_2)
results = model.fit()
print(results.summary())

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_61076\3347534445.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_2.replace([np.nan, np.inf, -np.inf], 0, inplace=True)


                            OLS Regression Results                            
Dep. Variable:        salario_semanal   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     44.71
Date:                Fri, 30 May 2025   Prob (F-statistic):           5.98e-20
Time:                        00:12:45   Log-Likelihood:                -48289.
No. Observations:                4412   AIC:                         9.658e+04
Df Residuals:                    4409   BIC:                         9.660e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
edad         720.0374     80.126      8.986      0.0

In [180]:
#salario_semanal en edad, edad2 y educ
X_3 = x_train_ocu[["edad", "edad2", "intercepto", "educ"]]
X_3.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

Y_3 = y_train_ocu["salario_semanal"]
Y_3.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

model = sm.OLS(Y_3, X_3)
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:        salario_semanal   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     31.87
Date:                Fri, 30 May 2025   Prob (F-statistic):           2.24e-20
Time:                        00:12:46   Log-Likelihood:                -48286.
No. Observations:                4412   AIC:                         9.658e+04
Df Residuals:                    4408   BIC:                         9.661e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
edad         691.5456     80.907      8.547      0.0

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_61076\4062105668.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_3.replace([np.nan, np.inf, -np.inf], 0, inplace=True)


In [181]:
#salario_semanal en edad,edad2, educ y mujer
X_4 = x_train_ocu[["edad", "edad2", "intercepto", "educ", "mujer"]]
X_4.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

Y_4 = y_train_ocu["salario_semanal"]
Y_4.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

model = sm.OLS(Y_4, X_4)
results = model.fit()
print(results.summary())


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_61076\3276446922.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_4.replace([np.nan, np.inf, -np.inf], 0, inplace=True)


                            OLS Regression Results                            
Dep. Variable:        salario_semanal   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     43.30
Date:                Fri, 30 May 2025   Prob (F-statistic):           1.09e-35
Time:                        00:12:46   Log-Likelihood:                -48248.
No. Observations:                4412   AIC:                         9.651e+04
Df Residuals:                    4407   BIC:                         9.654e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
edad         715.7028     80.276      8.916      0.0

In [182]:
#Regresión de salario_semanal en las variables del punto anterior y vp_dummy y lee_dummy
X_5 = x_train_ocu[["edad", "edad2", "intercepto", "educ", "mujer", "vp_dummy", "lee_dummy"]]
X_5.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

Y_5 = y_train_ocu["salario_semanal"]
Y_5.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

model = sm.OLS(Y_5, X_5)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:        salario_semanal   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     31.91
Date:                Fri, 30 May 2025   Prob (F-statistic):           8.55e-38
Time:                        00:12:47   Log-Likelihood:                -48240.
No. Observations:                4412   AIC:                         9.649e+04
Df Residuals:                    4405   BIC:                         9.654e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
edad         697.9497     80.355      8.686      0.0